In [1]:
# importing required libraries
import re
import os
import time
import urllib
import random
import requests
import unicodedata
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
urls_in = pd.read_csv('List of Urls.csv')
# urls_in

In [3]:
list_urla = []
list_urlb = []

for i in urls_in.urla:  
    list_urla.append(str(i))
    
for j in urls_in.urlb:  
    list_urlb.append(str(j))
    
dfa = pd.DataFrame(list_urla)
dfb = pd.DataFrame(list_urlb)
dfa.columns = ['urlA']
dfb.columns = ['urlB']

In [4]:
def kickScraper(url_list):
    slugs = []

    #extract slugs from url
    for url in url_list:
        slugs.append(re.search('/projects/(.*)\?', url).group(1))

    s = requests.Session()
    r = s.get("https://www.kickstarter.com")
    soup = BeautifulSoup(r.text, 'html.parser')
    xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]
    
    query = """
    query Campaign($slug: String!) {
      project(slug: $slug) {
        risks
        story(assetWidth: 680)
      }
    }"""
    
    text = pd.DataFrame(columns=['url_T', 'story_url_T', 'risk_url_T'])
    
    for slug in slugs:

        max_attempts = 10
        attempts = 0

        while attempts < max_attempts:
            # If not rate limited, break out of while loop and continue with the rest of the code
            if r.status_code != 429:
                break

            # If rate limited, wait and try again
            time.sleep((2 ** attempts) + random.random())
            attempts = attempts + 1
        
        r = s.post("https://www.kickstarter.com/graph",
            headers= {
                "x-csrf-token": xcsrf
            },
            json = {
                "operationName":"Campaign",
                "variables":{
                    "slug": slug
                },
                "query": query
            })
        
        max_attempts1 = 20
        attempts1 = 0
        while attempts1 < max_attempts1:
            # If not rate limited, break out of while loop and continue with the rest of the code
            if r.status_code != 429:
                break

            # If rate limited, wait and try again
            time.sleep((2 ** attempts1) + random.random())
            attempts1 = attempts1 + 1
            
        result = r.json()

        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        story_row = ''
        for i in soup.find_all('p'):
            story_row += i.get_text().replace("\xa0", "").replace("\n", "")

        risk_row = ''
        risk_row += result["data"]["project"]["risks"].replace("\r", "").replace("\n", "")
        
        inText = {'url_T': slug, 'story_url_T': story_row, 'risk_url_T': risk_row}
        text = text.append(inText, ignore_index=True)

    return text

In [5]:
url_listA = []
for each1 in dfa['urlA'][3640:3700]:
    url_listA.append(each1)
# url_listA

url_listB = []
for each2 in dfb['urlB'][3640:3700]:
    url_listB.append(each2)
# url_listB

In [6]:
resultA = kickScraper(url_listA)
resultA.columns = ['urlA', 'story_urlA', 'risk_urlA']
resultA

,urlA,story_urlA,risk_urlA
0,147673113/misting-water-bottle-hydrate-and-sta...,We have developed a very unique cap for bottle...,Launching this new product comes with its own ...
1,bestfarmhouse/farm-house-x-tables-with-a-twist,My name is Philip Dart and I enjoy making furn...,Challenges associated with my production of th...
2,billyjonesbluez/the-new-american-bluesman,,I don't see very many foreseeable obstacles if...
3,firefighterrobot/robotic-firefighter,The project is develop and produce a robot tha...,The common risks that are associated with this...
4,ecobeebox/natural-beekeeping-american-made-hon...,Backyard Urban Beekeeping Made Simple A good f...,"If delays occur, you will get a direct call or..."
5,164646305/terminal-testament-final-piece-of-th...,Christianity along with faith in God are fadin...,The book itself came together in pieces and at...
6,654351854/landladl-s-healthy-farm-to-people,My Name is Sabine and I belong to a new genera...,For three years I have lived by my farming and...
7,iconxlive/icon-x-live-the-next-geration-live-c...,During our testing of this concept we hired li...,The biggest challenge we're having is getting ...
8,1057253503/super-bowl-selfie,We are raising funds to give all football fans...,Possible obstacles:Technical difficulties - -I...
9,kilnadil/animation-short-for-solar-d-by-valentina,A 14 year old animator hopes to bring her Orig...,Risk of backing this Project is shirt producti...


In [7]:
resultB = kickScraper(url_listB)
resultB.columns = ['urlB', 'story_urlB', 'risk_urlB']
resultB

,urlB,story_urlB,risk_urlB
0,147673113/misting-water-bottle-cap,We have developed a very unique cap for bottle...,We don't anticipate any major challenges but o...
1,bestfarmhouse/farm-house-x-tables,My name is Philip Dart and I enjoy making furn...,With any business or mass production has risks...
2,billyjonesbluez/birth-of-a-nation-the-new-amer...,"""...he has a voice that will enchant you. ...h...",...I don't forsee any risks at this point.
3,firefighterrobot/robotic-firefighter-stand-alo...,The project is to develop and produce a robot ...,The common risks that are associated with this...
4,ecobeebox/eye-candy-equipment-for-making-honey...,Eco Bee Box began in 2012 as a simple idea for...,"Labor disputes, shipper strikes, natural disas..."
5,164646305/culminating-testament-of-the-holy-bible,We have lost faith in so many regions.Maybe it...,"Risks will simply if the medallions, bookmarks..."
6,654351854/landladl-s-virtueller-abhof-verkauf,"Gemüsekisterl gibt es, Käse, Honig, Getränke u...","Die letzten 5 Jahre haben gezeigt, dass das We..."
7,iconxlive/icon-x-live-the-next-generation-of-l...,iCon-X Live is a monetized web streaming platf...,Current challenges in our overall system capab...
8,1057253503/i-heart-trc-food-truck,IHeartTRC is raising funds to bring a culinary...,After we are funded some of the risks that we ...
9,kilnadil/kilnadil,IMDBhttps://www.imdb.com/title/tt12404188/rele...,Budget and talent


In [8]:
resultFinal = pd.concat([resultA, resultB], axis=1, join='inner')
# resultFinal

In [9]:
resultFinal['urla'] = dfa['urlA'].iloc[3640:3700].values
resultFinal['urlb'] = dfb['urlB'].iloc[3640:3700].values

resultFinal = resultFinal[['urla', 'urlA', 'story_urlA', 'risk_urlA', 'urlb', 'urlB', 'story_urlB', 'risk_urlB']]
resultFinal

,urla,urlA,story_urlA,risk_urlA,urlb,urlB,story_urlB,risk_urlB
0,https://www.kickstarter.com/projects/147673113...,147673113/misting-water-bottle-hydrate-and-sta...,We have developed a very unique cap for bottle...,Launching this new product comes with its own ...,https://www.kickstarter.com/projects/147673113...,147673113/misting-water-bottle-cap,We have developed a very unique cap for bottle...,We don't anticipate any major challenges but o...
1,https://www.kickstarter.com/projects/bestfarmh...,bestfarmhouse/farm-house-x-tables-with-a-twist,My name is Philip Dart and I enjoy making furn...,Challenges associated with my production of th...,https://www.kickstarter.com/projects/bestfarmh...,bestfarmhouse/farm-house-x-tables,My name is Philip Dart and I enjoy making furn...,With any business or mass production has risks...
2,https://www.kickstarter.com/projects/billyjone...,billyjonesbluez/the-new-american-bluesman,,I don't see very many foreseeable obstacles if...,https://www.kickstarter.com/projects/billyjone...,billyjonesbluez/birth-of-a-nation-the-new-amer...,"""...he has a voice that will enchant you. ...h...",...I don't forsee any risks at this point.
3,https://www.kickstarter.com/projects/firefight...,firefighterrobot/robotic-firefighter,The project is develop and produce a robot tha...,The common risks that are associated with this...,https://www.kickstarter.com/projects/firefight...,firefighterrobot/robotic-firefighter-stand-alo...,The project is to develop and produce a robot ...,The common risks that are associated with this...
4,https://www.kickstarter.com/projects/ecobeebox...,ecobeebox/natural-beekeeping-american-made-hon...,Backyard Urban Beekeeping Made Simple A good f...,"If delays occur, you will get a direct call or...",https://www.kickstarter.com/projects/ecobeebox...,ecobeebox/eye-candy-equipment-for-making-honey...,Eco Bee Box began in 2012 as a simple idea for...,"Labor disputes, shipper strikes, natural disas..."
5,https://www.kickstarter.com/projects/164646305...,164646305/terminal-testament-final-piece-of-th...,Christianity along with faith in God are fadin...,The book itself came together in pieces and at...,https://www.kickstarter.com/projects/164646305...,164646305/culminating-testament-of-the-holy-bible,We have lost faith in so many regions.Maybe it...,"Risks will simply if the medallions, bookmarks..."
6,https://www.kickstarter.com/projects/654351854...,654351854/landladl-s-healthy-farm-to-people,My Name is Sabine and I belong to a new genera...,For three years I have lived by my farming and...,https://www.kickstarter.com/projects/654351854...,654351854/landladl-s-virtueller-abhof-verkauf,"Gemüsekisterl gibt es, Käse, Honig, Getränke u...","Die letzten 5 Jahre haben gezeigt, dass das We..."
7,https://www.kickstarter.com/projects/iconxlive...,iconxlive/icon-x-live-the-next-geration-live-c...,During our testing of this concept we hired li...,The biggest challenge we're having is getting ...,https://www.kickstarter.com/projects/iconxlive...,iconxlive/icon-x-live-the-next-generation-of-l...,iCon-X Live is a monetized web streaming platf...,Current challenges in our overall system capab...
8,https://www.kickstarter.com/projects/105725350...,1057253503/super-bowl-selfie,We are raising funds to give all football fans...,Possible obstacles:Technical difficulties - -I...,https://www.kickstarter.com/projects/105725350...,1057253503/i-heart-trc-food-truck,IHeartTRC is raising funds to bring a culinary...,After we are funded some of the risks that we ...
9,https://www.kickstarter.com/projects/kilnadil/...,kilnadil/animation-short-for-solar-d-by-valentina,A 14 year old animator hopes to bring her Orig...,Risk of backing this Project is shirt producti...,https://www.kickstarter.com/projects/kilnadil/...,kilnadil/kilnadil,IMDBhttps://www.imdb.com/title/tt12404188/rele...,Budget and talent


In [10]:
resultFinal.to_csv('./TextCorpus3640-3700.csv', index=False)